In [1]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import f1_score
from joblib import Parallel, delayed
from sklearn.base import clone
from sklearn.ensemble import VotingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.utils import class_weight

import warnings

warnings.filterwarnings("ignore")

from matplotlib import rc

rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [2]:
from sklearn.preprocessing import LabelEncoder


def encode_data(df):
    df['idhogar'] = LabelEncoder().fit_transform(df['idhogar'])


def feature_importance(forest, X_train, display_results=True):
    ranked_list = []
    zero_features = []

    importances = forest.feature_importances_
    indices = np.argsort(importances)[::-1]

    if display_results:
        print("Feature ranking:")

    for f in range(X_train.shape[1]):
        if display_results:
            print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]) + " - " + X_train.columns[
                indices[f]])

        ranked_list.append(X_train.columns[indices[f]])

        if importances[indices[f]] == 0.0:
            zero_features.append(X_train.columns[indices[f]])

    return ranked_list, zero_features

In [3]:
def do_features(df):
    feats_div = [('children_fraction', 'r4t1', 'r4t3'),
                 ('working_man_fraction', 'r4h2', 'r4t3'),
                 ('all_man_fraction', 'r4h3', 'r4t3'),
                 ('human_density', 'tamviv', 'rooms'),
                 ('human_bed_density', 'tamviv', 'bedrooms'),
                 ('rent_per_person', 'v2a1', 'r4t3'),
                 ('rent_per_room', 'v2a1', 'rooms'),
                 ('mobile_density', 'qmobilephone', 'r4t3'),
                 ('tablet_density', 'v18q1', 'r4t3'),
                 ('mobile_adult_density', 'qmobilephone', 'r4t2'),
                 ('tablet_adult_density', 'v18q1', 'r4t2'),
                 ]

    feats_sub = [('people_not_living', 'tamhog', 'tamviv'),
                 ('people_weird_stat', 'tamhog', 'r4t3')]

    for f_new, f1, f2 in feats_div:
        df['fe_' + f_new] = (df[f1] / df[f2]).astype(np.float32)
    for f_new, f1, f2 in feats_sub:
        df['fe_' + f_new] = (df[f1] - df[f2]).astype(np.float32)

    aggs_num = {'age': ['min', 'max', 'mean'], 'escolari': ['min', 'max', 'mean']}

    aggs_cat = {'dis': ['mean']}
    for s_ in ['estadocivil', 'parentesco', 'instlevel']:
        for f_ in [f_ for f_ in df.columns if f_.startswith(s_)]:
            aggs_cat[f_] = ['mean', 'count']

    for name_, df_ in [('18', df.query('age >= 18'))]:
        df_agg = df_.groupby('idhogar').agg({**aggs_num, **aggs_cat}).astype(np.float32)
        df_agg.columns = pd.Index(['agg' + name_ + '_' + e[0] + "_" + e[1].upper() for e in df_agg.columns.tolist()])
        df = df.join(df_agg, how='left', on='idhogar')
        del df_agg

    df.drop(['Id'], axis=1, inplace=True)

    return df

In [4]:
def convert_OHE2LE(df):
    tmp_df = df.copy(deep=True)
    for s_ in ['pared', 'piso', 'techo', 'abastagua', 'sanitario', 'energcocinar', 'elimbasu',
               'epared', 'etecho', 'eviv', 'estadocivil', 'parentesco',
               'instlevel', 'lugar', 'tipovivi',
               'manual_elec']:
        if 'manual_' not in s_:
            cols_s_ = [f_ for f_ in df.columns if f_.startswith(s_)]
        elif 'elec' in s_:
            cols_s_ = ['public', 'planpri', 'noelec', 'coopele']

        sum_ohe = tmp_df[cols_s_].sum(axis=1).unique()

        if 0 in sum_ohe:
            print('The OHE in {} is incomplete. A new column will be added before label encoding'.format(s_))
            col_dummy = s_ + '_dummy'
            tmp_df[col_dummy] = (tmp_df[cols_s_].sum(axis=1) == 0).astype(np.int8)
            cols_s_.append(col_dummy)
            sum_ohe = tmp_df[cols_s_].sum(axis=1).unique()
            if 0 in sum_ohe:
                print("The category completion did not work")
        tmp_cat = tmp_df[cols_s_].idxmax(axis=1)
        tmp_df[s_ + '_LE'] = LabelEncoder().fit_transform(tmp_cat).astype(np.int16)
        if 'parentesco1' in cols_s_:
            cols_s_.remove('parentesco1')
        tmp_df.drop(cols_s_, axis=1, inplace=True)
    return tmp_df

In [5]:
train = pd.read_csv('../data/household/train.csv')
test = pd.read_csv('../data/household/test.csv')

test_ids = test.Id

In [6]:
def process_df(df_):
    encode_data(df_)
    return do_features(df_)


train = process_df(train)
test = process_df(test)

In [7]:
train['dependency'] = np.sqrt(train['SQBdependency'])
test['dependency'] = np.sqrt(test['SQBdependency'])

train.loc[train['edjefa'] == "no", "edjefa"] = 0
train.loc[train['edjefe'] == "no", "edjefe"] = 0
test.loc[test['edjefa'] == "no", "edjefa"] = 0
test.loc[test['edjefe'] == "no", "edjefe"] = 0

train.loc[(train['edjefa'] == "yes") & (train['parentesco1'] == 1), "edjefa"] = train.loc[
    (train['edjefa'] == "yes") & (train['parentesco1'] == 1), "escolari"]
train.loc[(train['edjefe'] == "yes") & (train['parentesco1'] == 1), "edjefe"] = train.loc[
    (train['edjefe'] == "yes") & (train['parentesco1'] == 1), "escolari"]

test.loc[(test['edjefa'] == "yes") & (test['parentesco1'] == 1), "edjefa"] = test.loc[
    (test['edjefa'] == "yes") & (test['parentesco1'] == 1), "escolari"]
test.loc[(test['edjefe'] == "yes") & (test['parentesco1'] == 1), "edjefe"] = test.loc[
    (test['edjefe'] == "yes") & (test['parentesco1'] == 1), "escolari"]

train.loc[train['edjefa'] == "yes", "edjefa"] = 4
train.loc[train['edjefe'] == "yes", "edjefe"] = 4

test.loc[test['edjefa'] == "yes", "edjefa"] = 4
test.loc[test['edjefe'] == "yes", "edjefe"] = 4

train['edjefe'] = train['edjefe'].astype("int")
train['edjefa'] = train['edjefa'].astype("int")
test['edjefe'] = test['edjefe'].astype("int")
test['edjefa'] = test['edjefa'].astype("int")

train['edjef'] = np.max(train[['edjefa', 'edjefe']], axis=1)
test['edjef'] = np.max(test[['edjefa', 'edjefe']], axis=1)

train['v2a1'] = train['v2a1'].fillna(0)
test['v2a1'] = test['v2a1'].fillna(0)

test['v18q1'] = test['v18q1'].fillna(0)
train['v18q1'] = train['v18q1'].fillna(0)

train['rez_esc'] = train['rez_esc'].fillna(0)
test['rez_esc'] = test['rez_esc'].fillna(0)

train.loc[train.meaneduc.isnull(), "meaneduc"] = 0
train.loc[train.SQBmeaned.isnull(), "SQBmeaned"] = 0

test.loc[test.meaneduc.isnull(), "meaneduc"] = 0
test.loc[test.SQBmeaned.isnull(), "SQBmeaned"] = 0

train.loc[(train.v14a == 1) & (train.sanitario1 == 1) & (train.abastaguano == 0), "v14a"] = 0
train.loc[(train.v14a == 1) & (train.sanitario1 == 1) & (train.abastaguano == 0), "sanitario1"] = 0

test.loc[(test.v14a == 1) & (test.sanitario1 == 1) & (test.abastaguano == 0), "v14a"] = 0
test.loc[(test.v14a == 1) & (test.sanitario1 == 1) & (test.abastaguano == 0), "sanitario1"] = 0

In [8]:
def train_test_apply_func(train_, test_, func_):
    test_['Target'] = 0
    xx = pd.concat([train_, test_])

    xx_func = func_(xx)
    train_ = xx_func.iloc[:train_.shape[0], :]
    test_ = xx_func.iloc[train_.shape[0]:, :].drop('Target', axis=1)

    del xx, xx_func
    return train_, test_

In [9]:
train, test = train_test_apply_func(train, test, convert_OHE2LE)

The OHE in techo is incomplete. A new column will be added before label encoding
The OHE in instlevel is incomplete. A new column will be added before label encoding
The OHE in manual_elec is incomplete. A new column will be added before label encoding


In [10]:
cols_2_ohe = ['eviv_LE', 'etecho_LE', 'epared_LE', 'elimbasu_LE',
              'energcocinar_LE', 'sanitario_LE', 'manual_elec_LE',
              'pared_LE']
cols_nums = ['age', 'meaneduc', 'dependency',
             'hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total',
             'bedrooms', 'overcrowding']


def convert_geo2aggs(df_):
    tmp_df = pd.concat([df_[(['lugar_LE', 'idhogar'] + cols_nums)],
                        pd.get_dummies(df_[cols_2_ohe], columns=cols_2_ohe)], axis=1)

    geo_agg = tmp_df.groupby(['lugar_LE', 'idhogar']).mean().groupby('lugar_LE').mean().astype(np.float32)
    geo_agg.columns = pd.Index(['geo_' + e for e in geo_agg.columns.tolist()])

    del tmp_df
    return df_.join(geo_agg, how='left', on='lugar_LE')


train, test = train_test_apply_func(train, test, convert_geo2aggs)

In [12]:
train['num_over_18'] = train[train.age >= 18].groupby('idhogar')['age'].transform("count")
train['num_over_18'] = train.groupby("idhogar")["num_over_18"].transform("max")
train['num_over_18'] = train['num_over_18'].fillna(0).astype(int)

test['num_over_18'] = test[test.age >= 18].groupby('idhogar')['age'].transform("count")
test['num_over_18'] = test.groupby("idhogar")["num_over_18"].transform("max")
test['num_over_18'] = test['num_over_18'].fillna(0).astype(int)


def extract_features(df):
    df['bedrooms_to_rooms'] = df['bedrooms'] / df['rooms']
    df['rent_to_rooms'] = df['v2a1'] / df['rooms']
    df['tamhog_to_rooms'] = df['tamhog'] / df['rooms']
    df['r4t3_to_tamhog'] = df['r4t3'] / df['tamhog']
    df['r4t3_to_rooms'] = df['r4t3'] / df['rooms']
    df['v2a1_to_r4t3'] = df['v2a1'] / df['r4t3']
    df['v2a1_to_r4t3'] = df['v2a1'] / (df['r4t3'] - df['r4t1'])
    df['hhsize_to_rooms'] = df['hhsize'] / df['rooms']
    df['rent_to_hhsize'] = df['v2a1'] / df['hhsize']
    df['rent_to_over_18'] = df['v2a1'] / df['num_over_18']
    df.loc[df.num_over_18 == 0, "rent_to_over_18"] = np.nan


extract_features(train)
extract_features(test)

train.replace([np.inf, -np.inf], np.nan, inplace=True)
test.replace([np.inf, -np.inf], np.nan, inplace=True)

train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [13]:
needless_cols = ['r4t3', 'tamhog', 'tamviv', 'hhsize', 'v18q', 'v14a', 'agesq', 'mobilephone', 'female', ]

instlevel_cols = [s for s in train.columns.tolist() if 'instlevel' in s]

needless_cols.extend(instlevel_cols)

train = train.drop(needless_cols, axis=1)
test = test.drop(needless_cols, axis=1)

In [14]:
def split_data(train, y, sample_weight=None, households=None, test_percentage=0.20, seed=None):
    train2 = train.copy()

    cv_hhs = np.random.choice(households, size=int(len(households) * test_percentage), replace=False)
    cv_idx = np.isin(households, cv_hhs)
    X_test = train2[cv_idx]
    y_test = y[cv_idx]

    X_train = train2[~cv_idx]
    y_train = y[~cv_idx]

    if sample_weight is not None:
        y_train_weights = sample_weight[~cv_idx]
        return X_train, y_train, X_test, y_test, y_train_weights

    return X_train, y_train, X_test, y_test

In [15]:
X = train.query('parentesco1==1')

y = X['Target'] - 1
X = X.drop(['Target'], axis=1)

np.random.seed(seed=None)

train2 = X.copy()

train_hhs = train2.idhogar

households = train2.idhogar.unique()
cv_hhs = np.random.choice(households, size=int(len(households) * 0.15), replace=False)

cv_idx = np.isin(train2.idhogar, cv_hhs)

X_test = train2[cv_idx]
y_test = y[cv_idx]

X_train = train2[~cv_idx]
y_train = y[~cv_idx]

X_train = train2
y_train = y

train_households = X_train.idhogar

In [16]:
y_train_weights = class_weight.compute_sample_weight('balanced', y_train, indices=None)

In [17]:
extra_drop_features = [
    'agg18_estadocivil1_MEAN',
    'agg18_estadocivil6_COUNT',
    'agg18_estadocivil7_COUNT',
    'agg18_parentesco10_COUNT',
    'agg18_parentesco11_COUNT',
    'agg18_parentesco12_COUNT',
    'agg18_parentesco1_COUNT',
    'agg18_parentesco2_COUNT',
    'agg18_parentesco3_COUNT',
    'agg18_parentesco4_COUNT',
    'agg18_parentesco5_COUNT',
    'agg18_parentesco6_COUNT',
    'agg18_parentesco7_COUNT',
    'agg18_parentesco8_COUNT',
    'agg18_parentesco9_COUNT',
    'geo_elimbasu_LE_4',
    'geo_energcocinar_LE_1',
    'geo_energcocinar_LE_2',
    'geo_epared_LE_0',
    'geo_hogar_mayor',
    'geo_manual_elec_LE_2',
    'geo_pared_LE_3',
    'geo_pared_LE_4',
    'geo_pared_LE_5',
    'geo_pared_LE_6',
    'num_over_18',
    'parentesco_LE',
    'rez_esc']

In [18]:
xgb_drop_cols = extra_drop_features + ["idhogar", 'parentesco1']

In [19]:
opt_parameters = {'max_depth': 35, 'eta': 0.1, 'silent': 0, 'objective': 'multi:softmax', 'min_child_weight': 1,
                  'num_class': 4, 'gamma': 2.0, 'colsample_bylevel': 0.9, 'subsample': 0.84, 'colsample_bytree': 0.88,
                  'reg_lambda': 0.40}

opt_parameters = {'max_depth': 35, 'eta': 0.15, 'silent': 1, 'objective': 'multi:softmax', 'min_child_weight': 2,
                  'num_class': 4, 'gamma': 2.5, 'colsample_bylevel': 1, 'subsample': 0.95, 'colsample_bytree': 0.85,
                  'reg_lambda': 0.35}


def evaluate_macroF1_lgb(predictions, truth):
    pred_labels = predictions.argmax(axis=1)
    truth = truth.get_label()
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', 1 - f1)


fit_params = {"early_stopping_rounds": 500,
              "eval_metric": evaluate_macroF1_lgb,
              "eval_set": [(X_train, y_train), (X_test, y_test)],
              'verbose': False,
              }


def learning_rate_power_0997(current_iter):
    base_learning_rate = 0.1
    min_learning_rate = 0.02
    lr = base_learning_rate * np.power(.995, current_iter)
    return max(lr, min_learning_rate)


fit_params['verbose'] = 50

In [20]:
np.random.seed(100)


def _parallel_fit_estimator(estimator1, X, y, sample_weight=None, threshold=True, **fit_params):
    estimator = clone(estimator1)

    if sample_weight is not None:
        X_train, y_train, X_test, y_test, y_train_weight = split_data(X, y, sample_weight, households=train_households)
    else:
        X_train, y_train, X_test, y_test = split_data(X, y, None, households=train_households)

    fit_params["eval_set"] = [(X_test, y_test)]

    if sample_weight is not None:
        if isinstance(estimator1, ExtraTreesClassifier) or isinstance(estimator1, RandomForestClassifier):
            estimator.fit(X_train, y_train)
        else:
            _ = estimator.fit(X_train, y_train, sample_weight=y_train_weight, **fit_params)
    else:
        if isinstance(estimator1, ExtraTreesClassifier) or isinstance(estimator1, RandomForestClassifier):
            estimator.fit(X_train, y_train)
        else:
            _ = estimator.fit(X_train, y_train, **fit_params)

    if not isinstance(estimator1, ExtraTreesClassifier) and not isinstance(estimator1,
                                                                           RandomForestClassifier) and not isinstance(
        estimator1, xgb.XGBClassifier):
        best_cv_round = np.argmax(estimator.evals_result_['validation_0']['mlogloss'])
        best_cv = np.max(estimator.evals_result_['validation_0']['mlogloss'])
        best_train = estimator.evals_result_['train']['macroF1'][best_cv_round]
    else:
        best_train = f1_score(y_train, estimator.predict(X_train), average="macro")
        best_cv = f1_score(y_test, estimator.predict(X_test), average="macro")
        print("Train F1:", best_train)
        print("Test F1:", best_cv)

    if threshold:
        if ((best_cv > 0.37) and (best_train > 0.75)) or ((best_cv > 0.44) and (best_train > 0.65)):
            return estimator
        else:
            print("Unacceptable!!! Trying again...")
            return _parallel_fit_estimator(estimator1, X, y, sample_weight=sample_weight, **fit_params)

    else:
        return estimator


class VotingClassifierLGBM(VotingClassifier):
    def fit(self, X, y, sample_weight=None, threshold=True, **fit_params):

        if isinstance(y, np.ndarray) and len(y.shape) > 1 and y.shape[1] > 1:
            raise NotImplementedError('Multilabel and multi-output'
                                      ' classification is not supported.')

        if self.voting not in ('soft', 'hard'):
            raise ValueError("Voting must be 'soft' or 'hard'; got (voting=%r)"
                             % self.voting)

        if self.estimators is None or len(self.estimators) == 0:
            raise AttributeError('Invalid `estimators` attribute, `estimators`'
                                 ' should be a list of (string, estimator)'
                                 ' tuples')

        if (self.weights is not None and
                len(self.weights) != len(self.estimators)):
            raise ValueError('Number of classifiers and weights must be equal'
                             '; got %d weights, %d estimators'
                             % (len(self.weights), len(self.estimators)))

        names, clfs = zip(*self.estimators)
        self._validate_names(names)

        n_isnone = np.sum([clf is None for _, clf in self.estimators])
        if n_isnone == len(self.estimators):
            raise ValueError('All estimators are None. At least one is '
                             'required to be a classifier!')

        self.le_ = LabelEncoder().fit(y)
        self.classes_ = self.le_.classes_
        self.estimators_ = []

        transformed_y = self.le_.transform(y)

        self.estimators_ = Parallel(n_jobs=self.n_jobs)(
            delayed(_parallel_fit_estimator)(clone(clf), X, transformed_y,
                                             sample_weight=sample_weight, threshold=threshold, **fit_params)
            for clf in clfs if clf is not None)

        return self

In [23]:
clfs = []
for i in range(15):
    clf = xgb.XGBClassifier(random_state=217 + i, n_estimators=300, learning_rate=0.15, n_jobs=4, **opt_parameters)

    clfs.append(('xgb{}'.format(i), clf))

vc = VotingClassifierLGBM(clfs, voting='soft')
del (clfs)

#Train the final model with learning rate decay
_ = vc.fit(X_train.drop(xgb_drop_cols, axis=1), y_train, sample_weight=y_train_weights, threshold=False, **fit_params)

clf_final = vc.estimators_[0]

AxisError: axis 1 is out of bounds for array of dimension 1

In [24]:
global_score = f1_score(y_test, clf_final.predict(X_test.drop(xgb_drop_cols, axis=1)), average='macro')
vc.voting = 'soft'
global_score_soft = f1_score(y_test, vc.predict(X_test.drop(xgb_drop_cols, axis=1)), average='macro')
vc.voting = 'hard'
global_score_hard = f1_score(y_test, vc.predict(X_test.drop(xgb_drop_cols, axis=1)), average='macro')

print('Validation score of a single LGBM Classifier: {:.4f}'.format(global_score))
print('Validation score of a VotingClassifier on 3 LGBMs with soft voting strategy: {:.4f}'.format(global_score_soft))
print('Validation score of a VotingClassifier on 3 LGBMs with hard voting strategy: {:.4f}'.format(global_score_hard))

NameError: name 'clf_final' is not defined

In [25]:
useless_features = []
drop_features = set()
counter = 0
for est in vc.estimators_:
    ranked_features, unused_features = feature_importance(est, X_train.drop(xgb_drop_cols, axis=1),
                                                          display_results=False)
    useless_features.append(unused_features)
    if counter == 0:
        drop_features = set(unused_features)
    else:
        drop_features = drop_features.intersection(set(unused_features))
    counter += 1

drop_features

set()

In [26]:
ranked_features = feature_importance(clf_final, X_train.drop(xgb_drop_cols, axis=1))

NameError: name 'clf_final' is not defined

In [28]:
et_drop_cols = ['agg18_age_MAX', 'agg18_age_MEAN', 'agg18_age_MIN', 'agg18_dis_MEAN',
                'agg18_escolari_MAX', 'agg18_escolari_MEAN', 'agg18_escolari_MIN',
                'agg18_estadocivil1_COUNT', 'agg18_estadocivil1_MEAN',
                'agg18_estadocivil2_COUNT', 'agg18_estadocivil2_MEAN',
                'agg18_estadocivil3_COUNT', 'agg18_estadocivil3_MEAN',
                'agg18_estadocivil4_COUNT', 'agg18_estadocivil4_MEAN',
                'agg18_estadocivil5_COUNT', 'agg18_estadocivil5_MEAN',
                'agg18_estadocivil6_COUNT', 'agg18_estadocivil6_MEAN',
                'agg18_estadocivil7_COUNT', 'agg18_estadocivil7_MEAN',
                'agg18_parentesco10_COUNT', 'agg18_parentesco10_MEAN',
                'agg18_parentesco11_COUNT', 'agg18_parentesco11_MEAN',
                'agg18_parentesco12_COUNT', 'agg18_parentesco12_MEAN',
                'agg18_parentesco1_COUNT', 'agg18_parentesco1_MEAN',
                'agg18_parentesco2_COUNT', 'agg18_parentesco2_MEAN',
                'agg18_parentesco3_COUNT', 'agg18_parentesco3_MEAN',
                'agg18_parentesco4_COUNT', 'agg18_parentesco4_MEAN',
                'agg18_parentesco5_COUNT', 'agg18_parentesco5_MEAN',
                'agg18_parentesco6_COUNT', 'agg18_parentesco6_MEAN',
                'agg18_parentesco7_COUNT', 'agg18_parentesco7_MEAN',
                'agg18_parentesco8_COUNT', 'agg18_parentesco8_MEAN',
                'agg18_parentesco9_COUNT', 'agg18_parentesco9_MEAN']

et_drop_cols.extend(["idhogar", "parentesco1", 'fe_rent_per_person', 'fe_rent_per_room',
                     'fe_tablet_adult_density', 'fe_tablet_density'])

In [29]:
ets = []
for i in range(10):
    rf = RandomForestClassifier(max_depth=None, random_state=217 + i, n_jobs=4, n_estimators=700,
                                min_impurity_decrease=1e-3, min_samples_leaf=2, verbose=0, class_weight="balanced")
    ets.append(('rf{}'.format(i), rf))

vc2 = VotingClassifierLGBM(ets, voting='soft')
_ = vc2.fit(X_train.drop(et_drop_cols, axis=1), y_train, threshold=False)

Train F1: 0.8971976052018816
Test F1: 0.4130162974280621
Train F1: 0.899344601612763
Test F1: 0.4355662026652593
Train F1: 0.8868585341751569
Test F1: 0.4064271389749889
Train F1: 0.9007040382372216
Test F1: 0.4473175901492684
Train F1: 0.8796763152108867
Test F1: 0.3987604043194966
Train F1: 0.8914965603461805
Test F1: 0.38763972806665137
Train F1: 0.8878426990059689
Test F1: 0.4187961132718414
Train F1: 0.8940621650131415
Test F1: 0.42426926358399086
Train F1: 0.8965505171736871
Test F1: 0.45484779430116773
Train F1: 0.8986130680097624
Test F1: 0.43800001933763977


In [30]:
vc2.voting = 'soft'
global_rf_score_soft = f1_score(y_test, vc2.predict(X_test.drop(et_drop_cols, axis=1)), average='macro')
vc2.voting = 'hard'
global_rf_score_hard = f1_score(y_test, vc2.predict(X_test.drop(et_drop_cols, axis=1)), average='macro')

print(
    'Validation score of a VotingClassifier on 3 LGBMs with soft voting strategy: {:.4f}'.format(global_rf_score_soft))
print(
    'Validation score of a VotingClassifier on 3 LGBMs with hard voting strategy: {:.4f}'.format(global_rf_score_hard))

Validation score of a VotingClassifier on 3 LGBMs with soft voting strategy: 0.8826
Validation score of a VotingClassifier on 3 LGBMs with hard voting strategy: 0.9126


In [31]:
vc2.voting = 'soft'
global_rf_score_soft = f1_score(y_test, vc2.predict(X_test.drop(et_drop_cols, axis=1)), average='macro')
vc2.voting = 'hard'
global_rf_score_hard = f1_score(y_test, vc2.predict(X_test.drop(et_drop_cols, axis=1)), average='macro')

print(
    'Validation score of a VotingClassifier on 3 LGBMs with soft voting strategy: {:.4f}'.format(global_rf_score_soft))
print(
    'Validation score of a VotingClassifier on 3 LGBMs with hard voting strategy: {:.4f}'.format(global_rf_score_hard))

Validation score of a VotingClassifier on 3 LGBMs with soft voting strategy: 0.8826
Validation score of a VotingClassifier on 3 LGBMs with hard voting strategy: 0.9126


In [32]:
useless_features = []
drop_features = set()
counter = 0
for est in vc2.estimators_:
    ranked_features, unused_features = feature_importance(est, X_train.drop(et_drop_cols, axis=1),
                                                          display_results=False)
    useless_features.append(unused_features)
    if counter == 0:
        drop_features = set(unused_features)
    else:
        drop_features = drop_features.intersection(set(unused_features))
    counter += 1

drop_features

{'parentesco_LE', 'rez_esc'}

In [33]:
def combine_voters(data, weights=[0.5, 0.5]):
    vc.voting = "soft"
    vc1_probs = vc.predict_proba(data.drop(xgb_drop_cols, axis=1))
    vc2.voting = "soft"
    vc2_probs = vc2.predict_proba(data.drop(et_drop_cols, axis=1))

    final_vote = (vc1_probs * weights[0]) + (vc2_probs * weights[1])
    predictions = np.argmax(final_vote, axis=1)

    return predictions

In [34]:
combo_preds = combine_voters(X_test, weights=[0.5, 0.5])
global_combo_score_soft = f1_score(y_test, combo_preds, average='macro')
global_combo_score_soft

0.034518828451882845

In [35]:
combo_preds = combine_voters(X_test, weights=[0.4, 0.6])
global_combo_score_soft = f1_score(y_test, combo_preds, average='macro')
global_combo_score_soft

0.034518828451882845

In [36]:
combo_preds = combine_voters(X_test, weights=[0.6, 0.4])
global_combo_score_soft = f1_score(y_test, combo_preds, average='macro')
global_combo_score_soft

0.034518828451882845

In [37]:
y_subm = pd.DataFrame()
y_subm['Id'] = test_ids

In [38]:
vc.voting = 'soft'
y_subm_lgb = y_subm.copy(deep=True)
y_subm_lgb['Target'] = vc.predict(test.drop(xgb_drop_cols, axis=1)) + 1

vc2.voting = 'soft'
y_subm_rf = y_subm.copy(deep=True)
y_subm_rf['Target'] = vc2.predict(test.drop(et_drop_cols, axis=1)) + 1

y_subm_ens = y_subm.copy(deep=True)
y_subm_ens['Target'] = combine_voters(test) + 1

AxisError: axis 1 is out of bounds for array of dimension 1

In [39]:
from datetime import datetime

now = datetime.now()

sub_file_lgb = 'submission_soft_XGB_{:.4f}_{}.csv'.format(global_score_soft, str(now.strftime('%Y-%m-%d-%H-%M')))
sub_file_rf = 'submission_soft_RF_{:.4f}_{}.csv'.format(global_rf_score_soft, str(now.strftime('%Y-%m-%d-%H-%M')))
sub_file_ens = 'submission_ens_{:.4f}_{}.csv'.format(global_combo_score_soft, str(now.strftime('%Y-%m-%d-%H-%M')))

y_subm_lgb.to_csv(sub_file_lgb, index=False)
y_subm_rf.to_csv(sub_file_rf, index=False)
y_subm_ens.to_csv(sub_file_ens, index=False)

NameError: name 'global_score_soft' is not defined